In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with
from bs4 import BeautifulSoup
import keyboard


In [2]:
site_1_baseURL = 'https://www.supabets.co.za/sports'
tab_name = ["1X2", "Over/Under 2.5 FT", "Both Teams to Score"]

In [3]:
# Set options for headless mode
options = webdriver.ChromeOptions()
options.add_argument("--headless")
# Set options for window size
options.add_argument("--window-size=1920,1200")
# Initialize the webdriver
driver = webdriver.Chrome(options)

# Navigate to the URL
driver.get(site_1_baseURL)
time.sleep(4)
# Get the HTML content using Selenium
html_content = driver.page_source

# Close the WebDriver
driver.quit()

# Parse the HTML content with BeautifulSoup
soup_entire = BeautifulSoup(html_content, 'html.parser')
sport_menu_items = soup_entire.select('ul.sportMenu > li.itemSport')

sport_menus = {}
for sport_menu_item in sport_menu_items:
    sport_menu_item_name = (sport_menu_item.select_one('div.head')['title']).upper()
    print(sport_menu_item_name)

    group_menu_items = sport_menu_item.select('ul.groupMenu > li.itemGroup')
    group_menus = {}
    for group_menu_item in group_menu_items:
        group_menu_item_name = (group_menu_item.select_one('div.head')['title']).upper()
        print("---->", group_menu_item_name)

        event_menu_items = group_menu_item.select('ul.eventMenu > li.itemEvent')
        event_menus = []
        for event_menu_item in event_menu_items:
            event_menu_item_name = event_menu_item.select_one('div.head')['title']
            print("-------->", event_menu_item_name)
            event_menus.append(event_menu_item_name)

        group_menus[group_menu_item_name] = event_menus
    sport_menus[sport_menu_item_name] = group_menus

print(sport_menus)


SOCCER
----> AFCON
--------> 2023 - Outright Winner
--------> 2023 - Group A Winner
--------> 2023 - Group B Winner
--------> 2023 - Group C Winner
--------> 2023 - Group D Winner
--------> 2023 - Group E Winner
--------> 2023 - Group F Winner
--------> Africa Cup of Nations Matches
----> ENGLAND
--------> Premier League
--------> Championship
--------> EFL Cup
--------> League One
--------> FA Cup
--------> League Two
--------> National League
--------> Championship - 2023/24 Winner
--------> League One 2023/2024 - Winner
--------> League Two - 2023/24 Winner
--------> Premier League - 2023/24 Relegation
--------> Premier League - 2023/24 Top  4
--------> Premier League - 2023/24 Winner
----> SPAIN
--------> LaLiga
--------> LaLiga 2
--------> Supercopa
----> GERMANY
--------> Bundesliga
--------> DFB Pokal
--------> 3. Liga
--------> Bundesliga - 2023/24 Winner
----> ITALY
--------> Serie A
--------> Serie B
--------> Serie C, Group A
--------> Serie C, Group B
--------> Serie C, Gro

In [4]:
# Function to recursively generate strings in the specified format
def generate_strings(data, prefix=''):
    result = []
    for key, value in data.items():
        current_prefix = f"{prefix}*{key}" if prefix else key
        
        if isinstance(value, list):
            result.extend([f"{current_prefix}*{item}" for item in value])
        elif isinstance(value, dict):
            result.extend(generate_strings(value, current_prefix))
    
    return result
# Updated replace_characters function
def replace_characters(input_string):
    replacements = [" - ", ", ", "/"]
    for replacement in replacements:
        input_string = input_string.replace(replacement, " ")
    # Replace " " with "-" and make lowercase
    input_string = input_string.replace(" ", "-").lower()
    input_string = input_string.replace("*", "/")
    return input_string

# Generate and print strings
strings = generate_strings(sport_menus)


In [5]:
matches_data = []
with open('data_1.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Site', 'Sports Menu', 'Country','Cup', 'Game', 'Date', 'Home', 'Draw', 'Away', 'O', 'U', 'Yes (GG)', 'No (NG)']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for string in strings:

            # Check if the home key is pressed
            if keyboard.is_pressed('home'):
                print("Home key pressed. Exiting loop.")
                break
            replace_characters(string)
            print((string))
            site1_subUrl = replace_characters(string)
            # print(site1_subUrl)
            site1_subUrl = f'{site_1_baseURL}/{site1_subUrl}'
            print(site1_subUrl)

            driver = webdriver.Chrome(options)
            driver.get(site1_subUrl)
            time.sleep(5)
        

            # Find all match containers
            match_containers = driver.find_elements(By.CSS_SELECTOR, '[is-event-item]')
            ul_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "CQ")))
            _date = ""

            # Loop through each match container
            for container in match_containers:
                try:
                    # print(container)
                    match_data = {}
                    match_data['Site'] = 'www.supabets.co.za'

                    infomations = string.split('*')
                    match_data['Sports Menu'] = infomations[0]
                    match_data['Country'] = infomations[1]
                    match_data['Cup'] = infomations[2]

                    
                    # Extract the game/teams
                    game = container.find_element(By.CSS_SELECTOR, '.subEvent.ng-binding').text
                    match_data['Game'] = game
                    # print("--------------------")
                    # print(game)
                    # print("xxxxxxxxxxxxxxxxxxxx")
                    
                    # Extract the date
                    date = container.find_element(By.CSS_SELECTOR, '.sepData.ng-binding').text
                    
                    if date == "":
                        date = _date
                    _date = date
                    match_data['Date'] = date
                    
                    for i in range(3):
                        # print(i)
                        li_element1 = ul_element.find_element(By.XPATH, f".//li[span[contains(text(), '{tab_name[i]}')]]")
                        li_element1.click()
                        time.sleep(2)
                        odds_elements = container.find_elements(By.CSS_SELECTOR, '.odd')
                        for odd_element in odds_elements:
                            odds_type = odd_element.find_element(By.CSS_SELECTOR, '.oddsType').get_attribute('title')
                            odds_value = odd_element.find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                            match_data[odds_type] = odds_value
                    print("xxxxxxxxxxxxxxxxxxxx")
                    print(match_data)
                    # Write data to CSV

                    writer.writerow(match_data)
                    matches_data.append(match_data)
                except Exception as e:
                    print(f"An error occurred: {e}")
                    continue


SOCCER*AFCON*2023 - Outright Winner
https://www.supabets.co.za/sports/soccer/afcon/2023-outright-winner
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//li[span[contains(text(), '1X2')]]"}
  (Session info: chrome-headless-shell=120.0.6099.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B8102142+3514994]
	(No symbol) [0x00007FF7B7D20CE2]
	(No symbol) [0x00007FF7B7BC76AA]
	(No symbol) [0x00007FF7B7C11860]
	(No symbol) [0x00007FF7B7C1197C]
	(No symbol) [0x00007FF7B7C064FC]
	(No symbol) [0x00007FF7B7C3602F]
	(No symbol) [0x00007FF7B7C063B6]
	(No symbol) [0x00007FF7B7C36490]
	(No symbol) [0x00007FF7B7C528F6]
	(No symbol) [0x00007FF7B7C35D93]
	(No symbol) [0x00007FF7B7C04BDC]
	(No symbol) [0x00007FF7B7C05C64]
	GetHandleVerifier [0x00007FF7B812E16B+3695259]
	GetHandleVerifier [0x00007FF7B8186737+